In [1]:
from google.colab import drive
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install -q -U keras-tuner
from kerastuner import HyperModel
from keras import models, layers,regularizers
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners.bayesian import BayesianOptimization
from tqdm import tqdm
import numpy as np

     |████████████████████████████████| 71kB 3.1MB/s 


In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_to_project = '/content/gdrive/My Drive/ML_HC_Project/' # Enter the path to your folder here
os.chdir(path_to_project)

In [4]:
ccle_exp = pd.read_csv("CCLE_expression.csv")
sample_info = pd.read_csv("sample_info.csv")
sanger_dose_response = pd.read_csv("sanger-dose-response.csv")
mapping = pd.read_csv('Ensembl_HGNC_map_042421.csv')

In [5]:
ccle_exp = ccle_exp.rename(columns={'Unnamed: 0':'DepMap_ID'})
ccle_exp.head()

,DepMap_ID,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),STPG1 (90529),NIPAL3 (57185),LAS1L (81887),ENPP4 (22875),SEMA3F (6405),CFTR (1080),ANKIB1 (54467),CYP51A1 (1595),KRIT1 (889),RAD52 (5893),BAD (572),LAP3 (51056),CD99 (4267),HS3ST1 (9957),AOC1 (26),WNT16 (51384),HECW1 (23072),MAD1L1 (8379),LASP1 (3927),SNX11 (29916),TMEM176A (55365),M6PR (4074),KLHL13 (90293),CYP26B1 (56603),ICA1 (3382),DBNDD1 (79007),ALS2 (57679),CASP10 (843),CFLAR (8837),...,PRSS50 (29122),CPHXL (105371346),AC131160.1 (55486),TPTEP2-CSNK1E (102800317),GNG14 (105372280),SLURP2 (432355),AC069544.2 (100421372),SCO2 (9997),C2orf81 (388963),PERCC1 (105371045),THSD8 (10535),LYNX1-SLURP2 (111188157),OR8B3 (390271),OR4F16 (81399),OR8B2 (26595),TMEM247 (388946),SMIM38 (107984345),OR8S1 (341568),OR4F29 (729759),EEF1AKMT4 (110599564),AC022414.1 (8622),TBCE (6905),SMIM41 (113523638),AC008397.1 (729966),GCSAML-AS1 (148824),CCDC39 (339829),EEF1AKMT4-ECE2 (110599583),AP000812.4 (220074),UPK3BL2 (107983993),AC093512.2 (112694756),ARHGAP11B (89839),AC004593.2 (1124),AC090517.4 (54816),AL160269.1 (11046),ABCF2-H2BE1 (114483834),POLR2J3 (548644),H2BE1 (114483833),AL445238.1 (647264),GET1-SH3BGR (106865373),AC113348.1 (102724657)
0,ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,5.053980,3.456806,4.415488,4.766595,2.280956,3.237258,0.000000,5.125982,6.636770,5.638364,3.881665,5.156639,4.775051,5.904966,0.097611,0.111031,0.042644,2.847997,3.336283,5.371210,4.313971,0.000000,7.536830,5.207893,2.965323,1.922198,2.049631,4.478972,2.077243,5.101818,...,0.748461,0.042644,1.782409,0.356144,0.650765,0.411426,2.895303,0.765535,1.731183,0.056584,0.695994,0.070389,0.0,0.070389,0.0,0.0,0.389567,0.0,0.070389,4.578335,0.000000,5.761019,0.0,0.000000,0.0,0.028569,0.000000,1.464668,5.234961,4.139961,1.214125,0.000000,0.111031,0.150560,1.427606,5.781884,0.0,0.000000,0.799087,0.000000
1,ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,3.832890,1.910733,3.374344,4.861955,3.625270,1.275007,0.028569,5.177121,7.130313,5.061776,3.023255,5.542258,6.305423,6.641546,0.084064,0.014355,0.275007,0.189034,2.903038,4.955127,4.421560,0.042644,7.133091,2.861955,0.124328,3.513491,4.056584,4.286142,0.333424,4.520422,...,0.000000,0.000000,1.799087,0.526069,0.000000,0.000000,4.202418,0.298658,0.765535,0.000000,4.173927,0.189034,0.0,0.014355,0.0,0.0,0.028569,0.0,0.014355,2.182692,0.042644,5.771357,0.0,0.000000,0.0,1.090853,0.000000,1.490570,0.941106,4.107688,1.835924,0.000000,0.310340,0.000000,0.807355,4.704319,0.0,0.000000,0.464668,0.070389
2,ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,3.775051,2.895303,3.613532,4.300856,0.799087,0.275007,0.042644,4.149747,5.655352,4.858976,2.675816,4.560715,6.170125,8.182245,0.389567,0.084064,0.084064,0.084064,5.733625,6.274262,4.407353,0.214125,7.361417,0.137504,0.454176,2.301588,3.317594,3.746313,1.232661,4.750607,...,0.014355,0.000000,0.992768,0.201634,0.000000,0.014355,3.209453,0.344828,0.555816,0.000000,2.952334,0.070389,0.0,0.014355,0.0,0.0,0.028569,0.0,0.014355,3.012569,0.000000,4.744699,0.0,0.000000,0.0,0.000000,0.000000,0.985500,1.124328,2.313246,1.823749,0.084064,0.176323,0.042644,1.384050,4.931683,0.0,0.028569,0.263034,0.000000
3,ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,4.294253,2.472488,4.573496,5.314697,3.488001,2.980025,0.028569,3.872829,6.176921,3.714795,2.726831,5.565293,5.230357,6.811728,2.657640,0.084064,0.124328,0.367371,4.161888,6.703211,3.541019,0.070389,7.208478,0.014355,1.124328,3.946731,3.237258,2.931683,2.797013,4.591560,...,0.000000,0.000000,0.641546,0.042644,0.000000,1.454176,2.438293,0.678072,1.117695,0.000000,0.000000,0.150560,0.0,0.000000,0.0,0.0,0.124328,0.0,0.000000,3.000000,0.000000,4.164304,0.0,0.454176,0.0,0.000000,0.000000,1.448901,0.201634,3.001802,0.871844,0.137504,0.263034,

In [6]:
sample_info.head()

,DepMap_ID,cell_line_name,stripped_cell_line_name,CCLE_Name,Alias,COSMICID,sex,source,Achilles_n_replicates,cell_line_NNMD,culture_type,culture_medium,cas9_activity,RRID,WTSI_Master_Cell_ID,sample_collection_site,primary_or_metastasis,primary_disease,Subtype,age,Sanger_Model_ID,depmap_public_comments,lineage,lineage_subtype,lineage_sub_subtype,lineage_molecular_subtype
0,ACH-000001,NIH:OVCAR-3,NIHOVCAR3,NIHOVCAR3_OVARY,OVCAR3,905933.0,Female,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0465,2201.0,ascites,Metastasis,Ovarian Cancer,"Adenocarcinoma, high grade serous",60.0,SIDM00105,NaN,ovary,ovary_adenocarcinoma,high_grade_serous,NaN
1,ACH-000002,HL-60,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,905938.0,Female,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0002,55.0,haematopoietic_and_lymphoid_tissue,Primary,Leukemia,"Acute Myelogenous Leukemia (AML), M3 (Promyelo...",35.0,SIDM00829,NaN,blood,AML,M3,NaN
2,ACH-000003,CACO2,CACO2,CACO2_LARGE_INTESTINE,"CACO2, CaCo-2",NaN,Male,ATCC,NaN,NaN,NaN,NaN,NaN,CVCL_0025,NaN,Colon,NaN,Colon/Colorectal Cancer,Adenocarcinoma,NaN,SIDM00891,NaN,colorectal,colorectal_adenocarcinoma,NaN,NaN
3,ACH-000004,HEL,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,907053.0,Male,DSMZ,2.0,-3.079202,Suspension,RPMI + 10% FBS,52.4,CVCL_0001,783.0,haematopoietic_and_lymphoid_tissue,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00594,NaN,blood,AML,M6,NaN
4,ACH-000005,HEL 92.1.7,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,Male,ATCC,2.0,-2.404409,Suspension,RPMI + 10% FBS,86.6,CVCL_2481,NaN,bone_marrow,NaN,Leukemia,"Acute Myelogenous Leukemia (AML), M6 (Erythrol...",30.0,SIDM00593,NaN,blood,AML,M6,NaN


In [7]:
sanger_dose_response.head()

,DATASET,COSMIC_ID,DRUG_ID,MIN_CONC,MAX_CONC,RMSE_PUBLISHED,Z_SCORE_PUBLISHED,IC50_PUBLISHED,AUC_PUBLISHED,ARXSPAN_ID,DRUG_NAME,BROAD_ID,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
0,GDSC1,683665,1,0.007813,2.0,0.022518,-0.192056,10.977393,0.982116,ACH-002270,ERLOTINIB,BRD-K70401845,0.992788,2.839376,-5.670993,0.514389,0.990834,NaN,0.000034,0.904675
1,GDSC1,684055,1,0.007813,2.0,0.031831,0.505823,23.133991,0.984820,ACH-002104,ERLOTINIB,BRD-K70401845,1.006405,0.028649,-0.186377,0.990054,0.997138,NaN,0.000057,0.028903
2,GDSC1,684057,1,0.007813,2.0,0.026047,1.280750,52.935278,0.985696,ACH-002106,ERLOTINIB,BRD-K70401845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GDSC1,684059,1,0.007813,2.0,0.110056,0.086028,14.774223,0.972701,ACH-002108,ERLOTINIB,BRD-K70401845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GDSC1,684062,1,0.007813,2.0,0.087010,-0.114395,11.926884,0.944463,ACH-002111,ERLOTINIB,BRD-K70401845,0.989580,0.075804,-12.222777,0.894027,0.933185,NaN,0.000623,0.777093


In [8]:
mapping.head()

,Ensembl_ID,HGNC_ID
0,ENSG00000000003,TSPAN6 (7105)
1,ENSG00000000005,TNMD (64102)
2,ENSG00000000419,DPM1 (8813)
3,ENSG00000000457,SCYL3 (57147)
4,ENSG00000000460,C1orf112 (55732)


In [9]:
cols_to_keep = ['DepMap_ID'] + list(mapping.HGNC_ID) 
ccle_chosen = ccle_exp[cols_to_keep]
ccle_chosen.head()

,DepMap_ID,TSPAN6 (7105),TNMD (64102),DPM1 (8813),SCYL3 (57147),C1orf112 (55732),FGR (2268),CFH (3075),FUCA2 (2519),GCLC (2729),NFYA (4800),STPG1 (90529),NIPAL3 (57185),LAS1L (81887),ENPP4 (22875),SEMA3F (6405),CFTR (1080),ANKIB1 (54467),CYP51A1 (1595),KRIT1 (889),RAD52 (5893),BAD (572),LAP3 (51056),CD99 (4267),HS3ST1 (9957),AOC1 (26),WNT16 (51384),HECW1 (23072),MAD1L1 (8379),LASP1 (3927),SNX11 (29916),TMEM176A (55365),M6PR (4074),KLHL13 (90293),CYP26B1 (56603),ICA1 (3382),DBNDD1 (79007),ALS2 (57679),CASP10 (843),CFLAR (8837),...,GCSAML-AS1 (148824),EPPK1 (83481),TSTD3 (100130890),CISD3 (284106),PPIAL4F (728945),PPIAL4E (730262),CT45A3 (441519),CT45A1 (541466),SPATA31A5 (727905),SPATA31A7 (26165),TBC1D3C (414060),OR4F29 (729759),SPANXB1 (728695),GRIFIN (402635),GAGE13 (645051),C2orf15 (150590),USP27X (389856),OR51B5 (282763),CT45A2 (728911),CT45A5 (441521),TBC1D3H (729877),NBPF15 (284565),WDR92 (116143),IQCJ-SCHIP1 (100505385),TYW1B (441250),ZNF286A (57335),OR13C5 (138799),PPIAL4A (653505),CCDC177 (56936),PADI6 (353238),CCL3L3 (414062),OR13C2 (392376),ZNF727 (442319),DUX4 (100288687),LYPD8 (646627),CCL15 (6359),POLR2J2 (246721),MTRNR2L12 (100462981),UGT2A1 (10941),ZNF8 (7554)
0,ACH-001113,4.990501,0.000000,7.273702,2.765535,4.480265,0.028569,1.269033,3.058316,6.483171,5.053980,3.456806,4.415488,4.766595,2.280956,3.237258,0.000000,5.125982,6.636770,5.638364,3.881665,5.156639,4.775051,5.904966,0.097611,0.111031,0.042644,2.847997,3.336283,5.371210,4.313971,0.000000,7.536830,5.207893,2.965323,1.922198,2.049631,4.478972,2.077243,5.101818,...,0.0,0.422233,0.028569,4.330558,0.097611,0.0,5.081936,6.491051,0.0,0.0,0.000000,0.070389,0.000000,0.0,0.0,1.257011,1.124328,0.000000,2.950468,4.426265,0.0,5.100137,3.166715,4.187451,1.389567,4.753284,0.0,0.000000,0.014355,0.000000,0.214125,0.0,0.000000,0.000000,0.042644,0.000000,2.344828,0.000000,0.0,2.097611
1,ACH-001289,5.209843,0.545968,7.070604,2.538538,3.510962,0.000000,0.176323,3.836934,4.200850,3.832890,1.910733,3.374344,4.861955,3.625270,1.275007,0.028569,5.177121,7.130313,5.061776,3.023255,5.542258,6.305423,6.641546,0.084064,0.014355,0.275007,0.189034,2.903038,4.955127,4.421560,0.042644,7.133091,2.861955,0.124328,3.513491,4.056584,4.286142,0.333424,4.520422,...,0.0,0.000000,0.111031,3.740928,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.056584,0.014355,0.000000,0.0,0.0,3.831877,2.687061,0.000000,0.000000,0.084064,0.0,3.504620,3.160275,1.298658,0.545968,5.245267,0.0,0.000000,0.226509,0.000000,0.000000,0.0,1.831877,0.028569,0.042644,0.000000,0.286881,0.310340,0.0,1.618239
2,ACH-001339,3.779260,0.000000,7.346425,2.339137,4.254745,0.056584,1.339137,6.724241,3.671293,3.775051,2.895303,3.613532,4.300856,0.799087,0.275007,0.042644,4.149747,5.655352,4.858976,2.675816,4.560715,6.170125,8.182245,0.389567,0.084064,0.084064,0.084064,5.733625,6.274262,4.407353,0.214125,7.361417,0.137504,0.454176,2.301588,3.317594,3.746313,1.232661,4.750607,...,0.0,0.000000,0.014355,4.732269,0.000000,0.0,0.000000,0.333424,0.0,0.0,0.000000,0.014355,3.157044,0.0,0.0,0.333424,1.321928,0.495695,0.000000,0.056584,0.0,2.680324,3.452859,4.926474,1.803227,4.311067,0.0,0.000000,0.028569,0.000000,0.176323,0.0,0.000000,0.000000,1.475085,0.000000,0.545968,0.189034,0.0,0.799087
3,ACH-001538,5.726831,0.000000,7.086189,2.543496,3.102658,0.000000,5.914565,6.099716,4.475733,4.294253,2.472488,4.573496,5.314697,3.488001,2.980025,0.028569,3.872829,6.176921,3.714795,2.726831,5.565293,5.230357,6.811728,2.657640,0.084064,0.124328,0.367371,4.161888,6.703211,3.541019,0.070389,7.208478,0.014355,1.124328,3.946731,3.237258,2.931683,2.797013,4.591560,...,0.0,0.321928,0.000000,3.697107,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.028569,0.000000,0.263034,0.0,0.0,2.518535,2.508429,1.682573,0.000000,0.028569,0.0,2.666757,3.040892,4.547820,1.604071,3.257011,0.0,0.014355,0.000000,0.042644,0.000000,0.0,0.475085,0.000000,0.863938,0.056584,0.275007,0.298658,0.0,2.111031
4,ACH-000242,7.465648,0.000000,6.435462,2.414136,3.864929,0.831877,7.198003,5.452530,7.112492

In [10]:
#### intersecting columns
len(set(list(ccle_chosen.DepMap_ID.unique())).intersection(list(sanger_dose_response.ARXSPAN_ID.unique())))

692

In [11]:
#### CHOSEN DRUGS

sanger_dose_response_filt = sanger_dose_response[(sanger_dose_response.DRUG_NAME.str.contains("BORTEZOMIB"))
                                                | (sanger_dose_response.DRUG_NAME.str.contains("LENALIDOMIDE"))
                                                ]

sanger_dose_response_filt = sanger_dose_response_filt.rename(columns={'ARXSPAN_ID':'DepMap_ID'})
sanger_dose_response_filt.head()

,DATASET,COSMIC_ID,DRUG_ID,MIN_CONC,MAX_CONC,RMSE_PUBLISHED,Z_SCORE_PUBLISHED,IC50_PUBLISHED,AUC_PUBLISHED,DepMap_ID,DRUG_NAME,BROAD_ID,upper_limit,ec50,slope,lower_limit,auc,log2.ic50,mse,R2
13983,GDSC1,683665,104,0.000078,0.02,0.046684,1.130432,0.021828,0.904474,ACH-002270,BORTEZOMIB,"BRD-K88510285, BRD-K50691590",0.999539,0.008465,-4.458199,0.623148,0.940851,NaN,0.000002,0.999863
13984,GDSC1,684055,104,0.000078,0.02,0.073249,1.096829,0.020736,0.878211,ACH-002104,BORTEZOMIB,"BRD-K88510285, BRD-K50691590",0.932104,0.140797,-0.942870,-1.053446,0.876455,NaN,0.000454,0.942287
13985,GDSC1,684057,104,0.000078,0.02,0.044637,0.771651,0.012617,0.805579,ACH-002106,BORTEZOMIB,"BRD-K88510285, BRD-K50691590",0.969575,0.003276,-2.188257,0.603120,0.849451,NaN,0.000091,0.995597
13986,GDSC1,684062,104,0.000078,0.02,0.017647,1.868700,0.067439,0.919632,ACH-002111,BORTEZOMIB,"BRD-K88510285, BRD-K50691590",0.998453,0.855216,-0.749972,-2.789605,0.946393,NaN,0.000127,0.972637
13987,GDSC1,684072,104,0.000078,0.02,0.053408,1.098944,0.020803,0.869446,ACH-000087,BORTEZOMIB,"BRD-K88510285, BRD-K50691590",0.977212,0.036327,-1.164436,-0.190464,0.904139,NaN,0.001364,0.919461


In [12]:
merged_df = sanger_dose_response_filt.merge(ccle_chosen, on= 'DepMap_ID')[list(ccle_chosen.columns)+['DRUG_NAME','IC50_PUBLISHED']]
merged_df['log(IC_50)'] = merged_df.IC50_PUBLISHED.apply(np.log10)
merged_df_bort = merged_df[merged_df.DRUG_NAME == 'BORTEZOMIB'].drop_duplicates()
merged_df_lenal = merged_df[merged_df.DRUG_NAME == 'LENALIDOMIDE'].drop_duplicates()
bort_labels = merged_df_bort['log(IC_50)']
lenal_labels = merged_df_lenal['log(IC_50)']
bort_data = merged_df_bort.drop(columns = ['DepMap_ID','IC50_PUBLISHED','DRUG_NAME','log(IC_50)'])
lenal_data = merged_df_lenal.drop(columns = ['DepMap_ID','IC50_PUBLISHED','DRUG_NAME','log(IC_50)'])
bort_train, bort_test, labels_bort_train, labels_bort_test = train_test_split(bort_data,bort_labels,random_state=42, train_size=0.7)
lenal_train, lenal_test, labels_lenal_train, labels_lenal_test = train_test_split(lenal_data,lenal_labels,random_state=42, train_size=0.7)

In [18]:
class RegressiveHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = models.Sequential()
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=1.0,
                    default=0.2,
                    step=0.1),
                    input_shape= input_shape
            )   
        )
        model.add(
            layers.Dense(
                units=hp.Int('units', 128,1024, 128, default=1024),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid','elu'],
                    default='relu'),
                kernel_regularizer= regularizers.l1(hp.Choice('reg_value',
                        values=[10.0**i for i in np.arange(0,-5,-1)]))
            )
        )

        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=1.0,
                    default=0.5,
                    step=0.1)
            )
        )

        model.add(
          layers.Dense(
              units=hp.Int('units', 32,128,32, default=138),
              activation=hp.Choice(
                  'dense_activation',
                  values=['relu', 'tanh', 'sigmoid','elu'],
                  default='relu'),
              kernel_regularizer= regularizers.l1(hp.Choice('reg_value',
                        values=[10.0**i for i in np.arange(0,-5,-1)]))
          )
        )

        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=1.0,
                    default=0.5,
                    step=0.1)
            )
        )

        model.add(
          layers.Dense(
              units=1,
              activation=hp.Choice(
                  'dense_activation',
                  values=['relu', 'tanh', 'sigmoid','elu'],
                  default='relu'),
              kernel_regularizer= regularizers.l1(hp.Choice('reg_value',
                        values=[10.0**i for i in np.arange(0,-5,-1)]))
          )
        )

        model.compile(
             optimizer = Adam(hp.Choice('learning_rate',
                          values=[1e-1,1e-2, 1e-3, 1e-4])),
                          loss='mse',metrics=['mse']
        )

        return model

##  Bortezomib

In [31]:
input_shape = (bort_train.shape[1],)
hypermodel_1 = RegressiveHyperModel(input_shape)

In [32]:

### using bayesian hyperparamter optimization
tuner_bo_1 = BayesianOptimization(
            hypermodel_1,
            objective='mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2
        )
tuner_bo_1.search(bort_train, labels_bort_train, epochs=10, validation_split=0.2, verbose=1)
best_model = tuner_bo_1.get_best_models(num_models=1)[0]

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [33]:
best_model

In [35]:
best_model.evaluate(bort_test, labels_bort_test)

8/8 [==============================] - 0s 12ms/step - loss: 3544.6482 - mse: 1.5128


[3544.648193359375, 1.5127743482589722]

## Lenalidomide

In [26]:
input_shape_2 = (lenal_train.shape[1],)
hypermodel_2 = RegressiveHyperModel(input_shape_2)

In [27]:
### using bayesian hyperparamter optimization
tuner_bo_2 = BayesianOptimization(
            hypermodel_2,
            objective='mse',
            max_trials=10,
            seed=42,
            executions_per_trial=2
        )
tuner_bo_2.search(lenal_train, labels_lenal_train, epochs=10, validation_split=0.2, verbose=1)
best_model_2 = tuner_bo_2.get_best_models(num_models=1)[0]

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [28]:
best_model_2

In [29]:
best_model_2.evaluate(lenal_test, labels_lenal_test)

7/7 [==============================] - 0s 11ms/step - loss: 3550.5042 - mse: 7.3689


[3550.48828125, 7.352900505065918]